In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sc
from IPython.display import Markdown, display
delta = 2
N = 64
beta = 1
a = 1
L = a*N
nMeas = 1000
nTherm = 500

def printmd(string):
    display(Markdown(string))

    
def gen_configuration(N):
    config = np.random.uniform(-10,10,size=N)
    config[0], config[N-1] = 0, 0
    return config

def Hamiltonian(config, N, a):
    hamiltonian_sum = 0
    for i in range(1, len(config)):
        hamiltonian_sum += (config[i]-config[i-1])**2
    return 1/a*hamiltonian_sum


def Metropolis(config, delta, beta, a):
    for i in range(0, N-1):
        config_update = config.copy()
        x = np.random.randint(1, N-1)
        config_update[x]+=np.random.uniform(-1,1)*delta
        if np.random.uniform(0,1)<=np.exp(-beta*(Hamiltonian(config_update, N, a)-Hamiltonian(config, N, a))):
            config = config_update
    
    return config
    
def numerical():
    config=gen_configuration(N)
    m_observed = []
    m_squared_observed = []
    energy_observed = []
    for i in range(nTherm):
        config = Metropolis(config, delta, beta, a)
    for i in range(nMeas):
        config_meas = Metropolis(config, delta, beta, a).copy()
        m_observed.append(a/L*np.sum(config_meas))
        m_squared_observed.append(a/L*np.sum(np.array(config_meas)**2))
        energy_observed.append(Hamiltonian(config_meas, N, a))
    return np.mean(m_observed),np.mean(m_squared_observed), np.mean(energy_observed)

In [2]:
m_num, m_squared_num, energy_num = numerical()
printmd("$\langle m_{num} \\rangle $=%.5f, $\langle m_{analytical}\\rangle$=%.5f "%(m_num,0))

$\langle m_{num} \rangle $=1.33598, $\langle m_{analytical}\rangle$=0.00000 

In [3]:
config = gen_configuration(N+1)
def fine_to_coarse(config):
     return [config[i] for i in range(len(config)) if i%2==0]

def linear_interpolation(config_coarse):
    linear_interpolation_list = []
    for i in range(len(config_coarse)):
        linear_interpolation_list.append(config_coarse[i])
        if i < len(config_coarse)-1: 
            linear_interpolation_list.append((config_coarse[i-1]+config_coarse[i+1])/2)
    return linear_interpolation_list
    
def coarse_to_fine(config, config_coarse): 
    return np.array(config)+np.array(linear_interpolation(config_coarse))

In [4]:
config_coarse = fine_to_coarse(config)
config_interpol = coarse_to_fine(config, config_coarse)